# IMPORTS

In [1]:
from __future__ import print_function
import random
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchinfo import summary
from tqdm import tqdm
from model import MnistNet_3


def seed_everything(seed=10):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(1)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

# MODEL

In [2]:
model = MnistNet_3()
summary(model, input_size=(1, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
MnistNet_3                               [1, 10]                   --
├─Sequential: 1-1                        [1, 8, 12, 12]            --
│    └─Conv2d: 2-1                       [1, 8, 26, 26]            72
│    └─BatchNorm2d: 2-2                  [1, 8, 26, 26]            16
│    └─ReLU: 2-3                         [1, 8, 26, 26]            --
│    └─Dropout: 2-4                      [1, 8, 26, 26]            --
│    └─Conv2d: 2-5                       [1, 16, 24, 24]           1,152
│    └─BatchNorm2d: 2-6                  [1, 16, 24, 24]           32
│    └─ReLU: 2-7                         [1, 16, 24, 24]           --
│    └─Dropout: 2-8                      [1, 16, 24, 24]           --
│    └─Conv2d: 2-9                       [1, 8, 24, 24]            128
│    └─MaxPool2d: 2-10                   [1, 8, 12, 12]            --
├─Sequential: 1-2                        [1, 16, 6, 6]             --
│    └─Conv

# DATALOADER

In [3]:
torch.manual_seed(1)
if use_cuda:
    torch.cuda.manual_seed(1)

batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.RandomAffine(degrees=15),  # scale=(0.95, 1.05)),       # 5, 10, 15
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=batch_size, shuffle=True, **kwargs)


# TRAINING & TESTING

In [4]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Epoch={epoch} Batch={batch_idx} loss={loss.item():.7f} Accuracy={100. * correct / len(train_loader.dataset):.2f}%')


def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.7f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# RUN

In [5]:
model = MnistNet_3().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.35, verbose=True)
criterion = nn.NLLLoss()

for epoch in range(15):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)
    scheduler.step()

Adjusting learning rate of group 0 to 5.0000e-02.


Epoch=0 Batch=468 loss=0.0506339 Accuracy=90.56%: 100%|██████████| 469/469 [00:10<00:00, 46.40it/s]



Test set: Average loss: 0.0007933, Accuracy: 9711/10000 (97.11%)

Adjusting learning rate of group 0 to 5.0000e-02.


Epoch=1 Batch=468 loss=0.0882496 Accuracy=97.37%: 100%|██████████| 469/469 [00:10<00:00, 46.03it/s]



Test set: Average loss: 0.0003369, Accuracy: 9859/10000 (98.59%)

Adjusting learning rate of group 0 to 5.0000e-02.


...


Test set: Average loss: 0.0001555, Accuracy: 9942/10000 (99.42%)

Adjusting learning rate of group 0 to 2.1437e-03.


Epoch=13 Batch=468 loss=0.0471064 Accuracy=98.91%: 100%|██████████| 469/469 [00:10<00:00, 45.59it/s]



Test set: Average loss: 0.0001560, Accuracy: 9941/10000 (99.41%)

Adjusting learning rate of group 0 to 2.1437e-03.


Epoch=14 Batch=468 loss=0.0130037 Accuracy=98.88%: 100%|██████████| 469/469 [00:10<00:00, 45.47it/s]



Test set: Average loss: 0.0001537, Accuracy: 9940/10000 (99.40%)

Adjusting learning rate of group 0 to 2.1437e-03.


# Target
- Reduce parameters to less than 7K
- Batch normalization and regularization
- Play around learning rate and scheduler configuration

# Result
- Parameters: 6,544
- Best Training Accuracy: 98.92%
- Best Test Accuracy: 99.45%

# Analysis
- Consistent test accuracy from 9th epoch >= 99.4%
- Train-test gap can be seen as train accuracy doesn't reach 99%